<a href="https://colab.research.google.com/github/Gabriel-Barroso-Ventura/virtual-assistant-pln/blob/main/Personal-assistant-PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Personal Assistant with Natural Language Processing**

In this project, we created a Python script that acts as a virtual assistant with the help of NLP (Natural Language Processing) techniques. It identifies voice commands to perform tasks.

Four tasks were implemented:

 - Search for a topic on Wikipedia;

 - Search for nearby pharmacies;

 - Open YouTube;

 - End the script.

Several tests were performed with the code, but it is important to note that it must be run on a computer. This is due to the need for a microphone to identify the user's speech. This device is not present in Google Colab.

Another important factor is that the script identifies all audio inputs on the computer, so there may be a need for changes that vary from machine to machine when choosing which device will capture the audio. Since the script may end up identifying speakers and sound drivers as microphone devices.

In [ ]:
import speech_recognition as sr
import pyttsx3
import wikipedia
import webbrowser
import geocoder
import folium
import requests
from geopy.geocoders import Nominatim
from geopy import distance
import time
import socket

class AssistenteVirtual:
    def __init__(self):
        # Inicializa o reconhecedor de fala
        self.reconhecedor = sr.Recognizer()

        # Lista microfones disponíveis (apenas para informação)
        print("\nMicrofones disponíveis:")
        microfones = sr.Microphone.list_microphone_names()
        for i, microfone in enumerate(microfones):
            print(f"[{i}] {microfone}")

        numero = int(input("\nEscolha o número do microfone que deseja usar: "))

        # Usa diretamente o microfone 2
        try:
            print(f"\nUsando microfone {numero}: {microfones[2]}")
            self.microfone = sr.Microphone(device_index=2)

            with self.microfone as fonte:
                print("Calibrando...")
                self.reconhecedor.adjust_for_ambient_noise(fonte, duration=1)
                print(f"Nível de energia: {self.reconhecedor.energy_threshold}")

                # Configura parâmetros de sensibilidade
                self.reconhecedor.dynamic_energy_threshold = True
                self.reconhecedor.pause_threshold = 0.5

        except Exception as e:
            raise Exception(f"Erro ao inicializar microfone {numero}: {str(e)}")

        # Inicializa o sintetizador de voz
        self.sintetizador = pyttsx3.init()
        wikipedia.set_lang("pt")

    def texto_para_fala(self, texto):
        """Converte texto em fala"""
        self.sintetizador.say(texto)
        self.sintetizador.runAndWait()

    def fala_para_texto(self):
        """Captura áudio do microfone e converte para texto"""
        with self.microfone as fonte:

            try:
                # Configura para ser mais sensível
                self.reconhecedor.pause_threshold = 0.5
                self.reconhecedor.energy_threshold = 1000

                # Espera por áudio
                audio = self.reconhecedor.listen(fonte, timeout=1, phrase_time_limit=None)
                texto = self.reconhecedor.recognize_google(audio, language='pt-BR')

                # Mostra o texto captado
                print(f"\rVocê: {texto}")
                return texto.lower()

            except sr.UnknownValueError:
                print("\rNão entendi o comando")
                return ""
            except sr.WaitTimeoutError:
                return ""
            except sr.RequestError:
                print("\nErro no serviço de reconhecimento de voz. Verifique sua conexão.")
                time.sleep(1)
                return ""

    def buscar_wikipedia(self, termo):
        """Realiza busca na Wikipedia"""
        print(f"Buscando na Wikipedia: {termo}")
        try:
            # Define um timeout para a busca
            socket.setdefaulttimeout(10)  # timeout de 10 segundos

            # Tenta buscar com diferentes variações do termo
            try:
                resultado = wikipedia.summary(termo, sentences=2)
            except wikipedia.exceptions.DisambiguationError as e:
                # Se houver múltiplos resultados, pega o primeiro
                resultado = wikipedia.summary(e.options[0], sentences=2)
            except wikipedia.exceptions.PageError:
                # Se não encontrar o termo exato, tenta uma busca
                resultados = wikipedia.search(termo, results=1)
                if resultados:
                    resultado = wikipedia.summary(resultados[0], sentences=2)
                else:
                    raise Exception("Nenhum resultado encontrado")

            print("\nResultado encontrado:", resultado)
            self.texto_para_fala(resultado)

        except Exception as e:
            erro = str(e)
            print(f"\nErro na busca: {erro}")
            if "Nenhum resultado encontrado" in erro:
                self.texto_para_fala("Não encontrei informações sobre isso na Wikipedia")
            else:
                self.texto_para_fala("Desculpe, ocorreu um erro ao buscar na Wikipedia")
        finally:
            # Reseta o timeout para o padrão
            socket.setdefaulttimeout(None)

    def abrir_youtube(self):
        """Abre o YouTube no navegador"""
        webbrowser.open("https://www.youtube.com")
        self.texto_para_fala("Abrindo YouTube")

    def encontrar_farmacia(self):
        """Encontra a farmácia mais próxima"""
        try:
            # Obtém localização atual
            print("Obtendo sua localização...")
            g = geocoder.ip('me')
            if not g.latlng:
                self.texto_para_fala("Não foi possível obter sua localização.")
                return

            latitude, longitude = g.latlng

            # Usa Nominatim com parâmetros mais específicos
            print("Procurando farmácias próximas...")
            geolocator = Nominatim(user_agent="assistente_virtual_1.0")

            # Busca farmácias usando Nominatim
            query = {
                'q': 'farmacia',
                'format': 'json',
                'limit': 5,
                'lat': latitude,
                'lon': longitude,
                'radius': 2000,
                'addressdetails': 1
            }

            response = requests.get(
                'https://nominatim.openstreetmap.org/search',
                params=query,
                headers={'User-Agent': 'assistente_virtual_1.0'}
            )

            # Verifica se a requisição foi bem sucedida
            if response.status_code != 200:
                self.texto_para_fala("Erro ao buscar farmácias. Serviço indisponível.")
                return

            try:
                farmacias = response.json()
            except:
                self.texto_para_fala("Erro ao processar dados das farmácias.")
                return

            if not farmacias:
                self.texto_para_fala("Não encontrei farmácias próximas.")
                return

            # Pega a primeira farmácia encontrada
            farmacia = farmacias[0]
            nome = farmacia.get('display_name', 'Farmácia').split(',')[0]
            lat_farm = float(farmacia['lat'])
            lon_farm = float(farmacia['lon'])

            # Calcula distância
            dist = distance.distance((latitude, longitude), (lat_farm, lon_farm)).km

            # Cria mapa
            mapa = folium.Map(location=[latitude, longitude], zoom_start=15)

            # Adiciona marcadores
            folium.Marker(
                [latitude, longitude],
                popup='Sua localização',
                icon=folium.Icon(color='red')
            ).add_to(mapa)

            folium.Marker(
                [lat_farm, lon_farm],
                popup=nome,
                icon=folium.Icon(color='green')
            ).add_to(mapa)

            # Salva e abre mapa
            mapa.save('mapa_farmacia.html')
            webbrowser.open('mapa_farmacia.html')

            self.texto_para_fala(f"Encontrei uma {nome} a {dist:.1f} quilômetros de distância")

        except Exception as e:
            print(f"Erro ao buscar farmácias: {str(e)}")
            self.texto_para_fala("Desculpe, ocorreu um erro ao buscar farmácias.")

    def executar(self):
        """Função principal que executa o assistente"""
        self.texto_para_fala("Olá! Como posso ajudar?")
        print("\nPronto para ouvir!")

        while True:
            comando = self.fala_para_texto()

            if not comando:  # Se não recebeu comando
                continue

            print(f"Processando comando: {comando}")

            try:
                comando = comando.lower().strip()

                if comando == 'wikipédia':
                    self.texto_para_fala("O que você quer pesquisar na Wikipedia?")
                    print("\nPronto para ouvir o termo de busca!")

                    # Espera pelo termo de busca
                    termo = self.fala_para_texto()
                    if termo:
                        print(f"Buscando na Wikipedia: {termo}")
                        self.buscar_wikipedia(termo)

                elif comando.startswith('wikipédia '):
                    # Remove a palavra 'wikipedia' e espaços extras
                    termo = comando.replace('wikipédia', '', 1).strip()
                    if termo:
                        print(f"Buscando na Wikipedia: {termo}")
                        self.buscar_wikipedia(termo)
                    else:
                        self.texto_para_fala("Por favor, diga o que você quer pesquisar na Wikipedia")

                elif 'youtube' in comando:
                    self.abrir_youtube()

                elif 'farmácia' in comando:
                    self.encontrar_farmacia()

                elif 'sair' in comando:
                    self.texto_para_fala("Até logo!")
                    break

                print("\nPronto para ouvir!")

            except Exception as e:
                print(f"Erro ao processar comando: {str(e)}")
                self.texto_para_fala("Desculpe, não consegui processar seu comando")
                print("\nPronto para ouvir!")

# Inicializa e executa o assistente
if __name__ == "__main__":
    assistente = AssistenteVirtual()
    assistente.executar()